Import Libraries

In [186]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Get the Understat_ID of the players from the below link

In [187]:

url = "https://github.com/vaastav/Fantasy-Premier-League/blob/master/data/2022-23/id_dict.csv?raw=true"
df_understat = pd.read_csv(url)

#Clean the column names
df_understat.columns = df_understat.columns.str.replace(' ', '')

#Get the column names
df_understat.columns

Index(['Understat_ID', 'FPL_ID', 'Understat_Name', 'FPL_Name'], dtype='object')

Get the Fpl Data which has been scrapped using FPL API

In [188]:
df_fpl = pd.read_excel('First_Draft.xlsx')

#Get the column names
df_fpl.columns

Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Total Points', 'Bonus', 'Minutes',
       'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Form'],
      dtype='object')

In [189]:
# Merge df_fpl and df_understat using the player_id column and _FPL_id column
df_fpl_understat = pd.merge(df_fpl, df_understat, left_on = "Player ID", right_on = "FPL_ID")

#Get the column names
df_fpl_understat.columns


Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Total Points', 'Bonus', 'Minutes',
       'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Form',
       'Understat_ID', 'FPL_ID', 'Understat_Name', 'FPL_Name'],
      dtype='object')

In [190]:
#keep only Understat_ID 
df_fpl_understat = df_fpl_understat[['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By',
    'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus', 'Minutes',
    'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Understat_ID']]

#convert Understat_ID to int
df_fpl_understat['Understat_ID'] = df_fpl_understat['Understat_ID'].astype(int)

pip install undetstatapi and import UnderstatClient

In [191]:
from understatapi import UnderstatClient
understat = UnderstatClient()

#Change Season to 2022 when the season starts on 5th August
league_player_data = understat.league(league="EPL").get_player_data(season="2021")

# convert to pandas dataframe
league_player_data = pd.DataFrame(league_player_data)
league_player_data.columns

Index(['id', 'player_name', 'games', 'time', 'goals', 'xG', 'assists', 'xA',
       'shots', 'key_passes', 'yellow_cards', 'red_cards', 'position',
       'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup'],
      dtype='object')

In [192]:
#convert columns to int
league_player_data['id'] = league_player_data['id'].astype(int)
league_player_data['xG'] = league_player_data['xG'].astype(float)
league_player_data['xA'] = league_player_data['xA'].astype(float)
league_player_data['npxG'] = league_player_data['npxG'].astype(float)
league_player_data['xGChain'] = league_player_data['xGChain'].astype(float)
league_player_data['xGBuildup'] = league_player_data['xGBuildup'].astype(float)
league_player_data['shots'] = league_player_data['shots'].astype(int)
league_player_data['key_passes'] = league_player_data['key_passes'].astype(int)


In [193]:
#merge with df_merged
final_df= pd.merge(df_fpl_understat, league_player_data, left_on = "Understat_ID", right_on = "id")

In [194]:
final_df.columns

Index(['Team', 'Player ID', 'First Name', 'Second Name', 'Web Name',
       'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus',
       'Minutes', 'Goals Scored', 'Assists', 'Clean Sheets', 'Status',
       'Understat_ID', 'id', 'player_name', 'games', 'time', 'goals', 'xG',
       'assists', 'xA', 'shots', 'key_passes', 'yellow_cards', 'red_cards',
       'position', 'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup'],
      dtype='object')

In [195]:
final_df = final_df[['Player ID', 'Understat_ID','Team', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By',
    'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus', 'games', 'Minutes',
    'Goals Scored', 'xG', 'Assists', 'xA', 'npxG', 'xGChain', 'xGBuildup', 'Clean Sheets', 'Status']]

In [196]:
#all numeric number to round to 2 decimal places
final_df = final_df.round(2)

In [197]:
final_df.columns

Index(['Player ID', 'Understat_ID', 'Team', 'First Name', 'Second Name',
       'Web Name', 'Position', 'Start Price', 'Current Price', 'Selected By',
       'Transfers In', 'Transfers Out', 'Form', 'Total Points', 'Bonus',
       'games', 'Minutes', 'Goals Scored', 'xG', 'Assists', 'xA', 'npxG',
       'xGChain', 'xGBuildup', 'Clean Sheets', 'Status'],
      dtype='object')

In [198]:
#rename columns
final_df.columns = ['Player ID', 'Understat_ID','Team', 'First Name', 'Second Name', 'Web Name',
    'Position', 'Start Price', 'Current Price', 'Selected By', 'Transfers In', 'Transfers Out', 'Form','Total Points', 'Bonus', 'Matches', 'Minutes', 'Goals Scored', 'xG', 'Assists', 'xA', 'npxG','xGChain', 'xGBuildup', 'Clean Sheets', 'Status']

In [199]:
#sort by Selected By and Total Points
final_df = final_df.sort_values(by=['Selected By', 'Total Points'], ascending=False)
final_df.head(10)

,Player ID,Understat_ID,Team,First Name,Second Name,Web Name,Position,Start Price,Current Price,Selected By,Transfers In,Transfers Out,Form,Total Points,Bonus,Matches,Minutes,Goals Scored,xG,Assists,xA,npxG,xGChain,xGBuildup,Clean Sheets,Status
22,28,5543,Arsenal,Gabriel,Fernando de Jesus,Jesus,Forward,0,8.0,70.8,0,0,0.0,120,16,28,1871,8,10.15,8,5.91,9.39,19.49,6.35,11,a
217,283,1250,Liverpool,Mohamed,Salah,Salah,Midfielder,0,13.0,58.8,0,0,0.0,265,29,35,2758,23,24.36,14,9.79,19.80,32.16,11.22,17,a
219,285,1791,Liverpool,Trent,Alexander-Arnold,Alexander-Arnold,Defender,0,7.5,57.0,0,0,0.0,208,32,32,2853,2,2.19,12,12.69,2.19,21.09,11.99,18,a
235,306,2379,Manchester City,João,Cancelo,Cancelo,Defender,0,7.0,44.9,0,0,0.0,201,27,36,3227,1,4.54,11,6.01,4.54,26.89,21.15,19,a
111,146,8067,Chelsea,Reece,James,James,Defender,0,6.0,39.7,0,0,0.0,141,26,26,1863,5,2.54,9,7.03,2.54,15.98,11.94,7,a
297,295,8204,Nottingham Forest,Neco,Williams,N.Williams,Defender,0,4.0,27.2,0,0,0.0,4,0,1,7,0,0.00,1,0.60,0.00,0.60,0.60,0,a
324,428,453,Tottenham,Son,Heung-min,Son,Midfielder,0,12.0,25.7,0,0,0.0,258,27,35,3009,23,16.99,10,7.85,16.99,29.62,10.34,18,a
323,427,647,Tottenham,Harry,Kane,Kane,Forward,0,11.5,25.6,0,0,0.0,192,23,37,3231,17,20.69,11,9.82,17.64,31.89,9.81,14,a
201,237,9493,Leeds United,Sam,Greenwood,Greenwood,Forward,0,4.5,25.5,0,0,0.0,15,0,7,231,0,0.23,2,0.43,0.23,1.23,0.67,0,a
227,293,10408,Liverpool,Luis,Díaz,Luis Díaz,Midfielder,0,8.0,24.4,0,0,0.0,64,6,13,957,4,4.28,3,2.18,4.28,8.96,3.45,6,a
